## 0. Install Dependecies

In [ ]:
pip install torch

**Set device to GPU if is available otherwise set device as cpu**

In [ ]:
import torch
# Check if GPU is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

**Import libraries**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

##1. Dataset##

- The Datatset used in this project was generated using the Mujoco simulator with three different configurations:
- 2D (2 joints)
- 2D (3 joints)
- 3D (5 joints)

The format of the data is in CSV format, including information about Joint angles, fingertip position, and orientation.


1.1. Visualise data from the simulator

In [ ]:
!head -5 logfiler2.csv

In [ ]:
!head -5 logfiler3.csv

j0;j1;j2;cos(j0);cos(j1);cos(j2);sin(j0);sin(j1);sin(j2);ft_x;ft_y;ft_qw;ft_qz
 0.055; -0.012;  0.072;  0.998;  1.000;  0.997;  0.055; -0.012;  0.072;  0.309;  0.022;  0.998;  0.057
 0.076; -0.017;  0.100;  0.997;  1.000;  0.995;  0.076; -0.017;  0.100;  0.308;  0.031;  0.997;  0.080
 0.135; -0.059;  0.194;  0.991;  0.998;  0.981;  0.135; -0.059;  0.193;  0.305;  0.050;  0.991;  0.135
 0.228; -0.110;  0.295;  0.974;  0.994;  0.957;  0.226; -0.109;  0.290;  0.297;  0.079;  0.979;  0.205


In [ ]:
!head -5 logfiler5.csv

j0;j1;j2;j3;j4;cos(j0);cos(j1);cos(j2);cos(j3);cos(j4);sin(j0);sin(j1);sin(j2);sin(j3);sin(j4);ft_x;ft_y;ft_z;ft_qw;ft_qx;ft_qy;ft_qz
 0.000;  0.000;  0.000;  0.000;  0.000;  1.000;  1.000;  1.000;  1.000;  1.000;  0.000;  0.000;  0.000;  0.000;  0.000;  0.000;  0.000;  0.590;  1.000;  0.000;  0.000;  0.000
 0.022; -0.005;  0.028;  0.016; -0.032;  1.000;  1.000;  1.000;  1.000;  0.999;  0.022; -0.005;  0.028;  0.016; -0.032;  0.011;  0.004;  0.590;  1.000; -0.016;  0.019;  0.011
 0.103;  0.005;  0.107;  0.017; -0.100;  0.995;  1.000;  0.994;  1.000;  0.995;  0.102;  0.005;  0.106;  0.017; -0.099;  0.041;  0.016;  0.587;  0.995; -0.053;  0.061;  0.054
 0.209;  0.067;  0.216;  0.013; -0.174;  0.978;  0.998;  0.977;  1.000;  0.985;  0.208;  0.067;  0.215;  0.013; -0.173;  0.100;  0.042;  0.573;  0.979; -0.101;  0.138;  0.116


1.2. Preprocess the data


2R Robot

In [ ]:
# Load dataset
data = pd.read_csv('logfiler2.csv', delimiter=';')

# Preprocessing: Extract inputs (joint angles and their trigonometric functions) and outputs (fingertip positions and quaternions)
X = data[['j0', 'j1', 'cos(j0)', 'cos(j1)', 'sin(j0)', 'sin(j1)']].values
y = data[['ft_x', 'ft_y', 'ft_qw', 'ft_qz']].values

# Normalize input features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

- Split the data into training and testing sets

In [ ]:
# Split data into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [ ]:
!head -5 X_train

In [ ]:
X_train.size()
y_train.size()

x_val.size()
y_val.size()

x_test.size()
y_test.size()

##2. Train Forward Kinematics Models##



### 2.1. Robot 2R

- Define the architecture of the model (Feedforward Neural Network) to learn forward kinematics.




- Define the loss function and optimizer

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the Neural Network model
class ForwardKinematicsModel(nn.Module):
    def __init__(self):
        super(ForwardKinematicsModel, self).__init__()
        # Define a simple feedforward network with 2 hidden layers
        self.fc1 = nn.Linear(2, 64)  # Input layer (2 joint angles)
        self.fc2 = nn.Linear(64, 64) # Hidden layer
        self.fc3 = nn.Linear(64, 2)  # Output layer (fingertip position)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize the model
model = ForwardKinematicsModel()

In [ ]:
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

- Train the models on joint angle inputs to predict fingertip positions.

In [ ]:
# Train the model
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()

    # Forward pass
    outputs = model(X_train.float())
    loss = criterion(outputs, Y_train.float())

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()        # Backpropagation
    optimizer.step()       # Update model parameters

    if (epoch + 1) % 100 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

In [ ]:
# Evaluate the model
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # No need to calculate gradients during inference
    predictions = model(X_test.float())
    test_loss = criterion(predictions, Y_test.float())
    print(f"Test Loss: {test_loss.item():.4f}")


- Hyperparameter Search

In [ ]:
learning_rates = [0.001, 0.01]
hidden_layer_sizes = [64, 128]

best_loss = float('inf')
best_params = None

for lr in learning_rates:
    for size in hidden_layer_sizes:
        # Define the model with new parameters
        model = ForwardKinematicsModel()
        optimizer = optim.Adam(model.parameters(), lr=lr)

        # Train the model for a few epochs (for hyperparameter tuning)
        for epoch in range(100):
            model.train()
            outputs = model(X_train.float())
            loss = criterion(outputs, Y_train.float())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Evaluate the model
        model.eval()
        with torch.no_grad():
            predictions = model(X_test.float())
            test_loss = criterion(predictions, Y_test.float())

        # Store the best parameters
        if test_loss.item() < best_loss:
            best_loss = test_loss.item()
            best_params = {'lr': lr, 'hidden_size': size}

print(f"Best Hyperparameters: {best_params}, Loss: {best_loss}")

Using Grid Se

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import mean_squared_error
import pandas as pd

# Define the Forward Kinematics Neural Network Model
class ForwardKinematicsModel(nn.Module):
    def __init__(self, hidden_size=64):
        super(ForwardKinematicsModel, self).__init__()
        # Define the network with flexible hidden layer size
        self.fc1 = nn.Linear(2, hidden_size)  # Input layer (2 joint angles)
        self.fc2 = nn.Linear(hidden_size, hidden_size)  # Hidden layer
        self.fc3 = nn.Linear(hidden_size, 2)  # Output layer (fingertip position)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Wrapper class for PyTorch model to use with scikit-learn GridSearchCV
class PyTorchRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, hidden_size=64, lr=0.001, epochs=100):
        self.hidden_size = hidden_size
        self.lr = lr
        self.epochs = epochs
        self.model = ForwardKinematicsModel(hidden_size=self.hidden_size)
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        self.criterion = nn.MSELoss()

    def fit(self, X, y):
        # Convert data to torch tensors
        X_train = torch.tensor(X, dtype=torch.float32)
        y_train = torch.tensor(y, dtype=torch.float32)

        # Training loop
        for epoch in range(self.epochs):
            self.model.train()
            self.optimizer.zero_grad()
            output = self.model(X_train)
            loss = self.criterion(output, y_train)
            loss.backward()
            self.optimizer.step()

        return self

    def predict(self, X):
        # Convert data to torch tensor and return predictions
        X_test = torch.tensor(X, dtype=torch.float32)
        self.model.eval()
        with torch.no_grad():
            predictions = self.model(X_test)
        return predictions.numpy()

# Load your dataset (replace this with your actual dataset)
df = pd.read_csv('path_to_data.csv', sep=';')
X = df[['j0', 'j1']].values  # Joint angles
y = df[['ft_x', 'ft_y']].values  # Fingertip positions

# Split dataset into training and test sets (80/20 split)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Set up hyperparameters to search
param_grid = {
    'hidden_size': [64, 128],       # Number of hidden units in the hidden layers
    'lr': [0.001, 0.01],            # Learning rates
    'epochs': [100, 200],           # Number of epochs to train
}

# Instantiate the PyTorch model wrapper for GridSearchCV
pytorch_model = PyTorchRegressor()

# Use GridSearchCV with the model wrapper
grid_search = GridSearchCV(estimator=pytorch_model, param_grid=param_grid, cv=3, verbose=2, n_jobs=1)

# Perform the grid search
grid_search.fit(X_train, y_train)

# Get the best hyperparameters and the best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Evaluate the best model on the test set
y_pred = best_model.predict(X_test)
test_loss = mean_squared_error(y_test, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test MSE: {test_loss:.4f}")


##3. Compare Jacobians##



3.1. Compute the Jacobian matrix for the learned forward kinematics using automatic differentiation.



In [ ]:
def compute_jacobian(model, input_tensor):
    # Use PyTorch's autograd to compute the Jacobian
    input_tensor = input_tensor.requires_grad_(True)
    output = model(input_tensor)

    # Compute Jacobian matrix (2x2 for 2 DOF robot)
    jacobian = torch.zeros(2, 2)
    for i in range(2):
        jacobian[i] = torch.autograd.grad(output[i], input_tensor, retain_graph=True)[0]
    return jacobian

# Example input (joint angles)
input_tensor = torch.tensor([0.1, 0.2])  # Example joint angles (2 DOF)
jacobian = compute_jacobian(model, input_tensor)
print(f"Jacobian:\n{jacobian}")


3.2. Compare the computed Jacobian with the analytical Jacobian for the 2-joint robot.

In [ ]:
import numpy as np

# Analytical Jacobian for 2-DOF robot (example)
def analytical_jacobian(theta):
    # This is an example, replace with the actual robot's kinematic equations
    # Here we assume a simple 2 DOF manipulator with planar joints
    J = np.array([[-np.sin(theta[0]) - np.sin(theta[0] + theta[1]), -np.sin(theta[0] + theta[1])],
                  [np.cos(theta[0]) + np.cos(theta[0] + theta[1]), np.cos(theta[0] + theta[1])]])
    return J

# Compute analytical Jacobian for a specific input
theta = [0.1, 0.2]
J_analytical = analytical_jacobian(theta)
print(f"Analytical Jacobian:\n{J_analytical}")

# Compare the learned Jacobian with the analytical Jacobian
# Convert learned Jacobian (from PyTorch) to a numpy array for comparison
J_learned = jacobian.detach().numpy()

# Compute the difference
jacobian_diff = np.linalg.norm(J_analytical - J_learned)
print(f"Difference between Analytical and Learned Jacobian: {jacobian_diff:.4f}")
